In [ ]:
!pip install numpy matplotlib solidspy scipy

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.7/92.7 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 8.8 MB/s eta 0:00:00
  Created wheel for solidspy: filename=solidspy-1.1.0.post1-py2.py3-none-any.whl size=32084 sha256=72f5ce7359a1190693e1545e91957c669f772df166a4fce0011c5674d63e12f9
  Stored in directory: /root/.cache/pip/wheels/c3/3f/b8/9fb202ed91fd7db93ba48b4129e7ae7db6fdb779f556a873ff
Successfully built solidspy


In [ ]:
import numpy as np
import solidspy.postprocesor as pos
import solidspy.uelutil as uel
from scipy.sparse import coo_matrix
from scipy.sparse.linalg import spsolve

from scipy.spatial.distance import cdist

def sparse_assem(elements, mats, neq, assem_op, nodes):
    """
    Assembles the global stiffness matrix
    using a sparse storing scheme

    Parameters
    ----------
    elements : ndarray (int)
      Array with the number for the nodes in each element.
    mats    : ndarray (float)
      Array with the material profiles.
    neq : int
      Number of active equations in the system.
    assem_op : ndarray (int)
      Assembly operator.
    uel : callable function (optional)
      Python function that returns the local stiffness matrix.
    kloc : ndarray
      Stiffness matrix of a single element

    Returns
    -------
    stiff : sparse matrix (float)
      Array with the global stiffness matrix in a sparse
      Compressed Sparse Row (CSR) format.
    """
    rows = []
    cols = []
    stiff_vals = []
    nels = elements.shape[0]
    for ele in range(nels):
        params = tuple(mats[elements[ele, 2], :])
        elcoor = nodes[elements[ele, -4:], 1:3]
        kloc, _ = uel.elast_quad4(elcoor, params)
        kloc_ = kloc * mats[elements[ele, 0], 2]
        ndof = kloc.shape[0]
        dme = assem_op[ele, :ndof]
        for row in range(ndof):
            glob_row = dme[row]
            if glob_row != -1:
                for col in range(ndof):
                    glob_col = dme[col]
                    if glob_col != -1:
                        rows.append(glob_row)
                        cols.append(glob_col)
                        stiff_vals.append(kloc_[row, col])

    stiff = coo_matrix((stiff_vals, (rows, cols)), shape=(neq, neq)).tocsr()

    return stiff

def optimality_criteria(nelx, nely, rho, d_c, g):
    """
    Optimality criteria method.

    Parameters
    ----------
    nelx : int
        Number of elements in x direction.
    nely : int
        Number of elements in y direction.
    rho : ndarray
        Array with the density of each element.
    d_c : ndarray
        Array with the derivative of the compliance.
    g : float
        Volume constraint.

    Returns
    -------
    rho_new : ndarray
        Array with the new density of each element.
    gt : float
        Volume constraint.
    """
    l1=0
    l2=1e9
    move=0.2
    rho_new=np.zeros(nelx*nely)
    while (l2-l1)/(l1+l2)>1e-3:
        lmid=0.5*(l2+l1)
        rho_new[:]= np.maximum(0.0,np.maximum(rho-move,np.minimum(1.0,np.minimum(rho+move,rho*np.sqrt(-d_c/lmid)))))
        gt=g+np.sum(((rho_new-rho)))
        if gt>0 :
            l1=lmid
        else:
            l2=lmid
    return (rho_new, gt)


def volume(els, length, height, nx, ny):
    """
    Volume calculation.

    Parameters
    ----------
    els : ndarray
        Array with models elements.
    length : ndarray
        Length of the beam.
    height : ndarray
        Height of the beam.
    nx : float
        Number of elements in x direction.
    ny : float
        Number of elements in y direction.

    Return
    ----------
    V: float
        Volume of a single element.
    """

    dy = length / nx
    dx = height / ny
    V = dx * dy * np.ones(els.shape[0])

    return V

def density_filter(centers, r_min, rho, d_rho):
    """
    Performe the sensitivity filter.

    Parameters
    ----------
    centers : ndarray
        Array with the centers of each element.
    r_min : float
        Minimum radius of the filter.
    rho : ndarray
        Array with the density of each element.
    d_rho : ndarray
        Array with the derivative of the density of each element.

    Returns
    -------
    densi_els : ndarray
        Sensitivity of each element with filter
    """
    dist = cdist(centers, centers, 'euclidean')
    delta = r_min - dist
    H = np.maximum(0.0, delta)
    densi_els = (rho*H*d_rho).sum(1)/(H.sum(1)*np.maximum(0.001,rho))

    return densi_els

def center_els(nodes, els):
    """
    Calculate the center of each element.

    Parameters
    ----------
    nodes : ndarray
        Array with models nodes.
    els : ndarray
        Array with models elements.

    Returns
    -------
    centers :
        Centers of each element.
    """
    centers = np.zeros((els.shape[0], 2))
    for el in els:
        n = nodes[el[-4:], 1:3]
        center = np.array([n[1,0] + (n[0,0] - n[1,0])/2, n[2,1] + (n[0,1] - n[2,1])/2])
        centers[int(el[0])] = center

    return centers

def sensi_el(nodes, mats, els, UC):
    """
    Calculate the sensitivity number for each element.

    Parameters
    ----------
    nodes : ndarray
        Array with models nodes
    mats : ndarray
        Array with models materials
    els : ndarray
        Array with models elements
    UC : ndarray
        Displacements at nodes

    Returns
    -------
    sensi_number : ndarray
        Sensitivity number for each element.
    """
    sensi_number = []
    for el in range(len(els)):
        params = tuple(mats[els[el, 2], :])
        elcoor = nodes[els[el, -4:], 1:3]
        kloc, _ = uel.elast_quad4(elcoor, params)
        node_el = els[el, -4:]
        U_el = UC[node_el]
        U_el = np.reshape(U_el, (8,1))
        a_i = U_el.T.dot(kloc.dot(U_el))[0,0]
        sensi_number.append(a_i)
    sensi_number = np.array(sensi_number)

    return sensi_number

def deformacionFill(nodes,elements, UC , factor,ms=1.0,cmap='Set1'):
    plt.figure(figsize=(9,6))
    coords = nodes[:,1:3] + UC * factor
    bcx = nodes[:,3]==-1
    bcy = nodes[:,4]==-1
    props = 0*coords[:,0]
    mats = np.unique(elements[:,2])
    i = 0
    for mat in mats:
        elIds =( elements[:,2] == mat )
        tris = elements[elIds,3:6]
        nodeIds = np.unique(tris)
        props[nodeIds] = mat
        i+=1
    # end for
    plt.tripcolor(coords[:,0],coords[:,1],elements[:,3:6],elements[:,2],cmap=cmap)
    plt.plot(coords[bcx,0],coords[bcx,1],"xr")
    plt.plot(coords[bcy,0],coords[bcy,1],"xr")
#    plt.triplot(coords[:,0],coords[:,1],elements[:,3:6],linewidth=0.1)
    # plt.tricontourf(coords[:,0],coords[:,1],elements[:,3:6],props,cmap='tab10')#, color ='C'+str(mat))
    plt.axis('image')
    plt.show()

    return


In [ ]:
import time
import random
import numpy as np
from scipy.sparse.linalg import spsolve
import solidspy.assemutil as ass # Solidspy 1.1.0

import matplotlib.pyplot as plt
from matplotlib import colors

# Start the timer
start_time = time.time()

np.seterr(divide='ignore', invalid='ignore')

# Initialize variables
length = 60
height = 60
nx = 100
ny= 100
penal = 3
Emax=20e9
Emin=0.0049
poisson_max = 0.15
poisson_min = 0.28

dirs = np.array([[0,-1]])
positions = np.array([[61,30]])
nodes, mats, els, loads, found_nodes = beam(L=length, H=height, nx=nx, ny=ny, dirs=dirs, positions=positions)

# Design variables initialization
change = 10 # Change in the design variable
g = 0 # Constraint
rho = 0.5 * np.ones(ny*nx, dtype=float) # Initialize the density
sensi_rho = np.ones(ny*nx) # Initialize the sensitivity
rho_old = rho.copy() # Initialize the density history
d_c = np.ones(ny*nx) # Initialize the design change
r_min = np.linalg.norm(nodes[0,1:3] - nodes[1,1:3]) * 4 # Radius for the sensitivity filter
centers = center_els(nodes, els) # Calculate centers
assem_op, bc_array, neq = ass.DME(nodes[:, -2:], els, ndof_el_max=8)

# Foundation initialization
els_nodes = els[:,-4:]
found_elements = [i for i, el in enumerate(els_nodes) if any(node in el for node in found_nodes)]
rho[found_elements] = 1

# Update material properties
density_gt_09 = rho > 0.95
density_lt_01 = rho < 0.6

mats[density_gt_09, 0] = Emax  # Update Young's modulus
mats[density_gt_09, 1] = poisson_max  # Update Poisson's ratio
mats[density_lt_01, 0] = Emin  # Update Young's modulus
mats[density_lt_01, 1] = poisson_min  # Update Poisson's ratio

iter = 0
for _ in range(100):
    iter += 1

    # Check convergence
    if change < 0.01:
        print('Convergence reached')
        break

    # Updata mats density
    mats[:,2] = 1e-6+rho**penal*(1.0-1e-6)
    # mats[:,2] = rho
    print(mats[:,2].min(), mats[:,2].max())

    # System assembly
    stiff_mat = sparse_assem(els, mats, neq, assem_op, nodes)
    rhs_vec = ass.loadasem(loads, bc_array, neq)

    # System solution
    disp = spsolve(stiff_mat, rhs_vec)
    UC = pos.complete_disp(bc_array, nodes, disp)
    _, S_nodes = pos.strain_nodes(nodes, els, mats[:,:2], UC)

    # Sensitivity analysis
    sensi_rho[:] = sensi_el(nodes, mats, els, UC)
    d_c[:] = (-penal*rho**(penal-1)*(Emax-Emin))*sensi_rho
    d_c[:] = density_filter(centers, r_min, rho, d_c)
    rho_old[:] = rho
    rho[:], g = optimality_criteria(nx, ny, rho, d_c, g)

    # density_gt_09 = mats[:, 2] > 0.95
    # density_lt_01 = mats[:, 2] < 0.05
    density_gt_09 = rho > 0.95
    density_lt_01 = rho < 0.05

    # Update values
    mats[density_gt_09, 0] = Emax  # Update Young's modulus
    mats[density_gt_09, 1] = 0.15  # Update Poisson's ratio

    mats[density_lt_01, 0] = Emin  # Update Young's modulus
    mats[density_lt_01, 1] = 0.28  # Update Poisson's ratio

    # Compute the change
    change = np.linalg.norm(rho.reshape(nx*ny,1)-rho_old.reshape(nx*ny,1),np.inf)
    print(change)

plt.ion()
fig,ax = plt.subplots()
ax.imshow(np.flipud(-mats[:,0].reshape(nx,ny)), cmap='gray', interpolation='none',norm=colors.Normalize(vmin=-1,vmax=0))
ax.set_title('Predicted')
fig.show()

plt.ion()
fig,ax = plt.subplots()
ax.imshow(np.flipud(-rho.reshape(nx,ny)), cmap='gray', interpolation='none',norm=colors.Normalize(vmin=-1,vmax=0))
ax.set_title('Predicted')
fig.show()

deformacionFill(nodes , els , UC , factor = 1.0 , cmap='cividis')

pos.fields_plot(els, nodes, UC , S_nodes=S_nodes)
